In [15]:
# Učitaj osnovne biblioteke...
import scipy as sp
import sklearn
import itertools
import cv2
import numpy as np

import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [16]:
def preprocess_image(image, max_width=640/2, max_height = 480/2):
    factor = 1
    if image.shape[0]>max_height:
        factor = max_height/float(image.shape[0])
    if image.shape[1]>max_width:
        f2 = max_width/float(image.shape[1])
        factor = f2 if f2 <factor else factor
    resized = cv2.resize(image,(0,0),fx=factor,fy=factor,interpolation=cv2.INTER_CUBIC) if factor <1 else image
    gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
    return resized,gray

In [17]:
def symmetry_score_for_pixel(scan_line, index, width, kernel):
    left = scan_line[index-width:index].astype(float)
    right = np.flipud(scan_line[index:index+width]).astype(float)
    diff = np.abs(left - right)
    return diff.dot(kernel)

In [18]:
def symmetry_scores_on_line(whole_image, height_index=400, width = 100):
    scan_line = whole_image[height_index,:]
    X = range(width,len(scan_line)-width)
    kernel = cv2.getGaussianKernel(width*2,0)[width:]*2
    return np.squeeze(np.array([symmetry_score_for_pixel(scan_line,index,width,kernel) for index in X]))

In [19]:
def symmetry_scores_map(image,width):#,verbose=False):
    img = np.zeros(image.shape[1] - width*2)
    for num in xrange(1,image.shape[0]):
        results = symmetry_scores_on_line(image,num,width)
        img = np.vstack((img,results))
    img = img.astype(np.float32)
    padding = np.zeros((img.shape[0],width))
    return np.hstack((padding,img,padding))#[:,w:img.shape[1]-w]

In [20]:
sigmas = logspace(0,10,11,base=np.sqrt(2))
print sigmas

[  1.           1.41421356   2.           2.82842712   4.           5.65685425
   8.          11.3137085   16.          22.627417    32.        ]


In [21]:

bgr_image = cv2.imread('tor2.jpg')

bgr_image, gray_image = preprocess_image(bgr_image)




def pretty_show(img):
    plt.figure(figsize=(20,20))
    plt.imshow(img,cmap='gray')
    plt.show()

#pretty_show(sym_map)

def calculate_dog_maps_mult(sym_map,sigmas):

    dog_maps = []

    kernel_size = 51
    sym_map_blured_old = cv2.GaussianBlur(sym_map,sigmaX=sigmas[0],ksize=(kernel_size,1)) 
    for sigma in sigmas[1:]:
        sym_map_blured = cv2.GaussianBlur(sym_map,sigmaX=sigma,ksize=(kernel_size,1))
        dog_map = sym_map_blured - sym_map_blured_old
        dog_map[dog_map<0] = 0
        dog_maps.append(dog_map)
        #pretty_show(dog_map)


        sym_map_blured_old = sym_map_blured


    dog_maps_mult=[]
    dog_map_old = dog_maps[0]
    for dog_map in dog_maps[1:]:
        dog_maps_mult.append(dog_map * dog_map_old)
        
        pretty_show(cv2.Canny(dog_map_mult,50,150,apertureSize = 3))
    return dog_maps_mult

width = 32

sym_map_horizontal = symmetry_scores_map(gray_image,width)
dog_maps_mult_horizontal = calculate_dog_maps_mult(sym_map_horizontal,sigmas)
sym_map_vertical = symmetry_scores_map(gray_image.T,width)
dog_maps_mult_vertical = calculate_dog_maps_mult(sym_map_vertical,sigmas)

suma_sumarum = dog_maps_mult_horizontal[0]*dog_maps_mult_vertical[0].T
for a,b in zip(dog_maps_mult_horizontal[1:],dog_maps_mult_vertical[1:]):
    
    plt.figure(figsize=(20,20))
    plt.subplot(121),plt.imshow(a,cmap='gray')   
    plt.subplot(122),plt.imshow(b.T,cmap='gray')
    plt.show()

    suma_sumarum+=a*b.T

pretty_show(suma_sumarum)



NameError: global name 'dog_map_mult' is not defined

In [22]:
def process_symmetry_scores(img)#,verbose = False):
    kernel_size= 11
    img = cv2.GaussianBlur(img,sigmaX=0,ksize=(kernel_size,1)) 
    img = cv2.Sobel(img,cv2.CV_32F,2,0,ksize=kernel_size)
    
    #if verbose:
    #    plt.figure(figsize=(10,10))
    #    plt.imshow(img,cmap='gray')
    #    plt.show()
    return img

SyntaxError: invalid syntax (<ipython-input-22-1c66c5d22ac9>, line 1)

In [23]:
def process_image(bgr_image,w,verbose = True):
   
    gray_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2GRAY)
    gray_image = cv2.GaussianBlur(gray_image,sigmaX=0,ksize=(3,3)) 
    
    if bgr_image.shape[1] -2*w<= 0 or bgr_image.shape[0]-2*w<=0:
        return
    
    processed_map_horizontal = symmetry_scores_in_image(gray_image,w,verbose)
    
    processed_map_vertical = symmetry_scores_in_image(gray_image.T,w,verbose)
    
    
    processed_map_horizontal[processed_map_horizontal<0]=0
    processed_map_vertical[processed_map_vertical<0]=0
    processed_map = processed_map_horizontal*processed_map_vertical.T
    processed_map = processed_map.astype(np.float32)
    processed_map[processed_map<np.max(processed_map)*0.4] = 0
    
    neighborhood_size = 5
    threshold = 0

    data = processed_map

    data_max = filters.maximum_filter(data, neighborhood_size)
    maxima = (data == data_max)
    data_min = filters.minimum_filter(data, neighborhood_size)
    diff = ((data_max - data_min) > threshold)
    maxima[diff == 0] = 0

    labeled, num_objects = ndimage.label(maxima)
    xy = np.array(ndimage.center_of_mass(data, labeled, range(1, num_objects+1)))
    if verbose:
        cv2.imshow("s",processed_map)
        plt.imshow(processed_map_horizontal,cmap='gray')
        plt.show()
        plt.imshow(processed_map_vertical,cmap='gray')
        plt.show()
        
        
        print "Feature count: "+str(len(xy))

        plt.imshow(processed_map,cmap='gray')    
        
        plt.scatter(xy[:, 1], xy[:, 0],marker='+',c='r')
        plt.show()
        
        plt.figure(figsize=(15,15.0/(640.0/480.0)))
        imgg = bgr_image.copy()
        imgg = cv2.resize(imgg,(0,0),fx=2,fy=2)
        for l in xy:
            cv2.circle(imgg,(int(l[1]*2),int(l[0]*2)),w*2,(255,0,0))


        plt.imshow(imgg)

        plt.axis([0,imgg.shape[1],imgg.shape[0],0])
        plt.scatter(xy[:, 1]*2, xy[:, 0]*2,marker='+',c='r')
        plt.show()
    
    #return process_features(gray_image,w,xy,bgr_image)

In [24]:

bgr_image = cv2.imread('notredame16/02.jpg')

bgr_image = preprocess_image(bgr_image)
process_image(bgr_image,32);

AttributeError: 'NoneType' object has no attribute 'shape'

In [25]:


filename = 'tor2.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,10,3,0.000001)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

In [26]:

img = cv2.imread('tor2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)

lines = cv2.HoughLines(edges,1,np.pi/180,200)
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite('houghlines3.jpg',img)

True